In [1]:
import cv2
import numpy as np
import os
import time
from matplotlib import pyplot as plt
import time
from HandTrackerRenderer import HandTrackerRenderer
from HandTrackerEdge import HandTracker
from FPS import FPS, now
from tensorflow import keras

In [2]:
#labelling  images
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
#importing variables
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('Collected_Data') 
# Actions that we try to detect
actions = np.array(['a','am','are','do','for','have','Hello','I','is','it','pretty','sell','shirt'
                    ,'skirt','so','the','to','wear','you','one','two','three','four','five','six'
                    ,'about','bad','look','think'])

# Thirty videos worth of data
no_sequences =60

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

label_map = {label:num for num, label in enumerate(actions)}

In [4]:
len(actions)

29

In [5]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        res = np.load(os.path.join(DATA_PATH, action, str(action)+'_'+ str(sequence)+'.npy'))
        sequences.append(res)
        labels.append(label_map[action])

In [6]:
res.shape

(60, 21, 3)

# Train Model

In [13]:
X = np.array(sequences)


In [14]:
y = to_categorical(labels).astype(int)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


In [16]:
X_train.shape

(1392, 60, 21, 3)

In [17]:
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],-1)
X_test = X_test.reshape(X_test.shape[0],X_train.shape[1],-1)
X_train.shape

(1392, 60, 63)

In [18]:
X_train.shape[1:3]

(60, 63)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [21]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [22]:
model = Sequential()

In [23]:
model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=(60, 63)))
model.add(LSTM(256, return_sequences=False, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 60, 128)           98304     
                                                                 
 lstm_1 (LSTM)               (None, 256)               394240    
                                                                 
 dense (Dense)               (None, 256)               65792     
                                                                 
 dense_1 (Dense)             (None, 512)               131584    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                                 
 dense_3 (Dense)             (None, 29)                7453      
                                                                 
Total params: 828,701
Trainable params: 828,701
Non-trai

In [25]:
model.compile(optimizer='Adagrad', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
44/44 [==============================] - 12s 207ms/step - loss: 3.3675 - categorical_accuracy: 0.0718
Epoch 2/2000
44/44 [==============================] - 9s 197ms/step - loss: 3.3658 - categorical_accuracy: 0.0812
Epoch 3/2000
44/44 [==============================] - 9s 203ms/step - loss: 3.3644 - categorical_accuracy: 0.0905
Epoch 4/2000
44/44 [==============================] - 9s 202ms/step - loss: 3.3632 - categorical_accuracy: 0.0826
Epoch 5/2000
44/44 [==============================] - 9s 206ms/step - loss: 3.3620 - categorical_accuracy: 0.0869
Epoch 6/2000
44/44 [==============================] - 9s 209ms/step - loss: 3.3611 - categorical_accuracy: 0.0898
Epoch 7/2000
44/44 [==============================] - 11s 245ms/step - loss: 3.3602 - categorical_accuracy: 0.0891
Epoch 8/2000
44/44 [==============================] - 11s 248ms/step - loss: 3.3594 - categorical_accuracy: 0.0869
Epoch 9/2000
44/44 [==============================] - 11s 243ms/step - loss: 3.3587 -

In [27]:
res = model.predict(X_test)
model.save('action29_LSTM_Relu.h5')


In [7]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [33]:
yhat = model.predict(X_test)

In [34]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [35]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[335,   0],
        [  0,  13]],

       [[330,   0],
        [  1,  17]],

       [[333,   1],
        [  2,  12]],

       [[339,   2],
        [  1,   6]],

       [[334,   2],
        [  0,  12]],

       [[337,   1],
        [  0,  10]],

       [[338,   0],
        [  1,   9]],

       [[332,   5],
        [  2,   9]],

       [[336,   1],
        [  4,   7]],

       [[337,   0],
        [  1,  10]],

       [[335,   0],
        [  0,  13]],

       [[340,   0],
        [  0,   8]],

       [[331,   0],
        [  1,  16]],

       [[333,   2],
        [  0,  13]],

       [[337,   1],
        [  0,  10]],

       [[340,   1],
        [  0,   7]],

       [[335,   1],
        [  1,  11]],

       [[335,   0],
        [  1,  12]],

       [[336,   0],
        [  1,  11]],

       [[340,   1],
        [  1,   6]],

       [[331,   0],
        [  2,  15]],

       [[333,   0],
        [  0,  15]],

       [[339,   0],
        [  0,   9]],

       [[331,   1],
        [  0, 

In [91]:
accuracy_score(ytrue, yhat)


NameError: name 'accuracy_score' is not defined

In [7]:
sign_model = keras.models.load_model('action29_LSTM_Relu.h5')

In [8]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [9]:
def extractData(hands) :
    rh_data = np.zeros((21, 3))
    lh_data = np.zeros((21, 3))
    for hand in hands : 
        Label = hand.label
        if Label == 'left' : 
            lh_data = hand.norm_landmarks 
        elif Label == 'right' : 
            rh_data = hand.norm_landmarks
    return lh_data,rh_data


    

In [51]:
nouns = ['shirt', 'skirt']
pronouns = ['it','you']
adjectives = ['pretty', 'bad']
verbs = ['am','are','do','have','is','sell','wear','look','think']
adverbs = []
prepositions = ['to','about']
conjunctions = ['so']
articles = ['a','the']
personals = ['I']
numbers = ['one','two','three','four','five','six']
checklist = 0
stop = False

def gramcheck(predicted_sign,now_sign) :
    global checklist
    global stop
    if len(now_sign) != 0 :
        now_sign = now_sign[0]
    if now_sign == [] :
        return True
    if (now_sign in personals) and (predicted_sign in verbs) :
        checklist+=1
        return True
    if (now_sign in nouns) and (predicted_sign in verbs)  :  
        checklist+=1
        return True
    if (now_sign in pronouns) and (predicted_sign in verbs) :
        checklist+=1
        return True
    if (now_sign in adjectives) and (predicted_sign in adjectives or predicted_sign in nouns) :
        if checklist >= 2 :
            checklist = 0
            stop = True
            return '.'
        else :  
            checklist+=1
            return True 
    if (now_sign in verbs) and (predicted_sign in articles or predicted_sign in adjectives or predicted_sign in numbers) :
        if predicted_sign in articles or predicted_sign in numbers :
            checklist+=1
            return True
        if predicted_sign in adjectives :
            if checklist >= 1 :
                checklist = 0
                stop = True
                return '.'
            else :
                return False
    if (now_sign in articles) and (predicted_sign in nouns) :
        if checklist >= 2 :
            checklist = 0
            stop = True
            return '.'
        else :  
            checklist+=1
            return True
    if (now_sign in numbers) and (predicted_sign in nouns or predicted_sign in adjectives) :
        if predicted_sign in adjectives:
            checklist+=1
            return True
        if predicted_sign in nouns :
            if checklist >= 2 :
                checklist = 0
                stop = True
                return '.'
            else :
                checklist+=1
                return True
        else :
            return False
    else :
        return False
        

In [52]:
tracker = HandTracker(solo = False)

renderer = HandTrackerRenderer(
        tracker=tracker,
        output=None)

Palm detection blob     : C:\Users\Windows 10\Documents\GitHub\SeniorProject\models\palm_detection_sh4.blob
Landmark blob           : C:\Users\Windows 10\Documents\GitHub\SeniorProject\models\hand_landmark_lite_sh4.blob
PD post processing blob : C:\Users\Windows 10\Documents\GitHub\SeniorProject\custom_models\PDPostProcessing_top2_sh1.blob
Sensor resolution: (1920, 1080)
Internal camera FPS set to: 36
Internal camera image size: 1152 x 648 - pad_h: 252
Creating pipeline...
Creating Color Camera...
Creating Palm Detection pre processing image manip...
Creating Palm Detection Neural Network...
Creating Palm Detection post processing Neural Network...
Creating Hand Landmark pre processing image manip...
Creating Hand Landmark Neural Network (2 threads)...
Pipeline created.
Pipeline started - USB speed: HIGH


In [57]:
#Threshold = [are,for]
lh = []
rh = []
sentence = []
threshold = 0.8
now_sign=[]
font=cv2.FONT_HERSHEY_SIMPLEX
color=(0,255,0)
size = 1
thickness=2
while True:
    frame, hands, bag = tracker.next_frame()
    if frame is None: break
    # Draw hands
    hand = renderer.draw(frame, hands, bag)
    key = renderer.waitKey(delay=1)
    lh_data,rh_data = extractData(hands)
    lh.append(lh_data)
    rh.append(rh_data)
    lh_used=lh[-30:]
    rh_used=rh[-30:]
    lrh = np.concatenate([lh_used,rh_used])
    lrh_used = lrh.reshape(lrh.shape[0],-1)
    cv2.putText(frame,f"data = {len(lrh)}",(20,50),font,size,color,thickness)
    # predictions logic
    if len(lrh) == 60:
        res = sign_model.predict(np.expand_dims(lrh_used, axis=0))[0]
        predicted_sign = actions[np.argmax(res)]
        confidence = res[np.argmax(res)]
        print(predicted_sign,now_sign,checklist,stop)
        if (confidence > threshold) & (predicted_sign not in now_sign)  :
            if stop == True :
                print(predicted_sign)
                print(confidence)
                stop=False
                now_sign=[actions[np.argmax(res)]]
            if gramcheck(predicted_sign,now_sign) == True :
                print(predicted_sign)
                print(confidence)
                now_sign=[actions[np.argmax(res)]]
            elif gramcheck(predicted_sign,now_sign) == '.' :
                print(predicted_sign)
                print(confidence)
                now_sign=[actions[np.argmax(res)]]
        lh = []
        rh = []
    if key == 27 or key == ord('q'):
        break
renderer.exit()

I [] 0 False
a [] 0 False
a
0.9758044
a ['a'] 0 False
sell ['a'] 0 False
skirt ['a'] 0 False
skirt
0.8205332
it ['skirt'] 1 False
for ['skirt'] 1 False
wear ['skirt'] 1 False
wear
0.8981276
wear ['wear'] 2 False
I ['wear'] 2 False
a ['wear'] 2 False
a
0.9770585
a ['a'] 3 False
think ['a'] 3 False
Hello ['a'] 3 False
Hello ['a'] 3 False
it ['a'] 3 False
it ['a'] 3 False
it ['a'] 3 False
skirt ['a'] 3 False
about ['a'] 1 True
I ['a'] 1 True
I ['a'] 1 True
Hello ['a'] 1 True
Hello
0.9099416
Hello ['Hello'] 1 False
it ['Hello'] 1 False
it ['Hello'] 1 False
I ['Hello'] 1 False
I ['Hello'] 1 False
I ['Hello'] 1 False
I ['Hello'] 1 False


In [50]:
tracker.exit()